This notebook computes step contact classification accuracy.

#  Input parameters here

In [1]:
# # PARAMETERS # # 
BASE_DIR = '/content/drive/My Drive/CS230_FinalReport/' # your base directory
MASK_VALUE = 999.
MAX_TIMESTEPS = 67
EXP_TYPE = 'with_physicsinputs'
# # # # # # # # # # 

# Load libraries and other set up

In [2]:
# Mount Drive to access data.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import os
import matplotlib.pyplot as plt

In [4]:
# Load in predictions and true labels
Y_pred = np.load(file = os.path.join(BASE_DIR, 'models', EXP_TYPE + '_lstm_stepclass', 'testset_predictions.npy'), allow_pickle = True)
Y_true = np.load(file=os.path.join(BASE_DIR, 'data', 'processed', 'normalized', 'with_physicsinputs', 'Y_test_stepclass.npy'), allow_pickle = True)

In [5]:
# Check overall accuracies
num_trials = Y_pred.shape[0]
R_accuracies = np.zeros(num_trials)
L_accuracies = np.zeros(num_trials)

for trial in range(num_trials):
  curr_pred = Y_pred[trial,:,:]
  curr_true = Y_true[trial,:,:]

  # Get min time step
  if MASK_VALUE in curr_true:
    min_timestep = np.min(np.nonzero(curr_true[:,0] == MASK_VALUE))
  else:
    min_timestep = MAX_TIMESTEPS

  # Get non-masked values
  curr_pred = curr_pred[0:min_timestep,:]
  curr_true = curr_true[0:min_timestep,:]

  # Get predicted labels based on probabilities
  curr_pred = np.where(curr_pred >= 0.5, 1, 0)

  # Calc and store accuracies
  R_accuracies[trial] = np.sum(curr_pred[:,0] == curr_true[:,0]) / min_timestep
  L_accuracies[trial] = np.sum(curr_pred[:,1] == curr_true[:,1]) / min_timestep

# Get average accuracies
print(f"average right leg step classification accuracy: {np.round(np.mean(R_accuracies),2)}")
print(f"average left leg step classification accuracy: {np.round(np.mean(L_accuracies),2)}")

average right leg step classification accuracy: 0.9
average left leg step classification accuracy: 0.93


In [10]:
# Compute per task accuracies
Y_pred_oagr = Y_pred[0:700,:,:]
Y_true_oagr = Y_true[0:700,:,:]
Y_pred_acl  = Y_pred[700:805,:,:]
Y_true_acl  = Y_true[700:805,:,:]

# OAGR: we can just loop over all trials as usual since all walking
num_trials = Y_pred_oagr.shape[0]
R_accuracies = np.zeros(num_trials)
L_accuracies = np.zeros(num_trials)

for trial in range(num_trials):
  curr_pred = Y_pred_oagr[trial,:,:]
  curr_true = Y_true_oagr[trial,:,:]

  # Get min time step
  if MASK_VALUE in curr_true:
    min_timestep = np.min(np.nonzero(curr_true[:,0] == MASK_VALUE))
  else:
    min_timestep = MAX_TIMESTEPS

  # Get non-masked values
  curr_pred = curr_pred[0:min_timestep,:]
  curr_true = curr_true[0:min_timestep,:]

  # Get predicted labels based on probabilities
  curr_pred = np.where(curr_pred >= 0.5, 1, 0)

  # Calc and store accuracies
  R_accuracies[trial] = np.sum(curr_pred[:,0] == curr_true[:,0]) / min_timestep
  L_accuracies[trial] = np.sum(curr_pred[:,1] == curr_true[:,1]) / min_timestep

# Get average accuracies
#print(f"average right leg step classification accuracy for all walking trials: {np.round(np.mean(R_accuracies),2)}")
#print(f"average left leg step classification accuracy for all walking trials: {np.round(np.mean(L_accuracies),2)}")
print(f"average right leg step classification accuracy for all walking trials: {np.mean(R_accuracies)}")
print(f"average left leg step classification accuracy for all walking trials: {np.mean(L_accuracies)}")

# ACL: reshape to get per task accuracies
num_trials = 7 * 3 # num trials per task
R_accuracies = np.zeros(shape=(5, num_trials)) # 5 tasks x num trials per task
L_accuracies = np.zeros(shape=(5, num_trials))

# Reshape
Y_pred_acl_reshaped = np.reshape(Y_pred_acl, (7,5,3,Y_pred_acl.shape[1], Y_pred_acl.shape[2])) # subjs x tasks x trials x timesteps x outputs
Y_true_acl_reshaped = np.reshape(Y_true_acl, (7,5,3,Y_true_acl.shape[1], Y_true_acl.shape[2]))

for task in range(5):
  task_pred = Y_pred_acl_reshaped[:,task,:,:,:]
  task_true = Y_true_acl_reshaped[:,task,:,:,:]

  for trial in range(num_trials):
    curr_pred = np.reshape(task_pred, (num_trials, MAX_TIMESTEPS, 2))
    curr_true = np.reshape(task_true, (num_trials, MAX_TIMESTEPS, 2))
    curr_pred = curr_pred[trial,:,:]
    curr_true = curr_true[trial,:,:]

    # Get min time step
    if MASK_VALUE in curr_true:
      min_timestep = np.min(np.nonzero(curr_true[:,0] == MASK_VALUE))
    else:
      min_timestep = MAX_TIMESTEPS

    # Get non-masked values
    curr_pred = curr_pred[0:min_timestep,:]
    curr_true = curr_true[0:min_timestep,:]

    # Get predicted labels based on probabilities
    curr_pred = np.where(curr_pred >= 0.5, 1, 0)

    # Calc and store accuracies
    R_accuracies[task, trial] = np.sum(curr_pred[:,0] == curr_true[:,0]) / min_timestep
    L_accuracies[task, trial] = np.sum(curr_pred[:,1] == curr_true[:,1]) / min_timestep

    # debugging
    #if task == 2:
      #print(f"R: {R_accuracies[task, trial] }")
      #print(f"L: {L_accuracies[task, trial] }")

# Get average accuracies
tasks = ['cutting','dj','lldj','rldj','unant_cut']
for task in range(5):
  #print(f"average right leg step classification accuracy for all {tasks[task]} trials: {np.round(np.mean(R_accuracies[task, :]),2)}")
  #print(f"average left leg step classification accuracy for all {tasks[task]} trials: {np.round(np.mean(L_accuracies[task, :]),2)}")
  print(f"average right leg step classification accuracy for all {tasks[task]} trials: {np.mean(R_accuracies[task, :])}")
  print(f"average left leg step classification accuracy for all {tasks[task]} trials: {np.mean(L_accuracies[task, :])}")

average right leg step classification accuracy for all walking trials: 0.8931079126148707
average left leg step classification accuracy for all walking trials: 0.9275144150843718
average right leg step classification accuracy for all cutting trials: 0.9422669067627051
average left leg step classification accuracy for all cutting trials: 0.9556386805930102
average right leg step classification accuracy for all dj trials: 1.0
average left leg step classification accuracy for all dj trials: 0.993061471751403
average right leg step classification accuracy for all lldj trials: 1.0
average left leg step classification accuracy for all lldj trials: 0.9989417989417989
average right leg step classification accuracy for all rldj trials: 1.0
average left leg step classification accuracy for all rldj trials: 0.9808191808191808
average right leg step classification accuracy for all unant_cut trials: 0.9496273150408489
average left leg step classification accuracy for all unant_cut trials: 0.9586142

In [16]:
# # Plot the 100% accuracy tasks (jumping) # #

# Load true GRF signals
Y_true_reg = np.load(file=os.path.join(BASE_DIR, 'models', 'with_physicsinputs', 'transformed_preds', 'Y_true_3d.npy'))
# Only get the ACL data and reshape
Y_true_reg = Y_true_reg[700:805,:,:]
Y_true_reg = np.reshape(Y_true_reg, (7,5,3,Y_true_reg.shape[1],Y_true_reg.shape[2]))

# Loop over jumping tasks only
acl_tasks = ['dj','lldj','rldj']
coords = ['x', 'y', 'z']
Y_true_reg = Y_true_reg[:, 1:4, :, :, :]
Y_true_class = Y_true_acl_reshaped[:, 1:4, :, :, :]
Y_pred_class = Y_pred_acl_reshaped[:, 1:4, :, :, :]

for subj in range(7):
  for task in range(3):
    for trial in range(3):
      
      # get data for curr trial
      curr_Y_true_reg = Y_true_reg[subj, task, trial, :, :] # timesteps x 12 signals
      curr_Y_true_class = Y_true_class[subj, task, trial, :, :] # timesteps x r and l leg classes
      curr_Y_pred_class = Y_pred_class[subj, task, trial, :, :]

      # make fig
      fig, axs = plt.subplots(2,2)
      
      # get min timestep
      if MASK_VALUE in curr_Y_true_class:
        min_timestep = np.min(np.nonzero(curr_Y_true_class[:,0] == MASK_VALUE))
      else:
        min_timestep = MAX_TIMESTEPS

      curr_Y_true_reg = curr_Y_true_reg[0:min_timestep, :]
      curr_Y_true_class = curr_Y_true_class[0:min_timestep, :]
      curr_Y_pred_class = np.where(curr_Y_pred_class[0:min_timestep, :] >= 0.5, 1, 0)

      # plot right leg GRF
      for coord in range(3):
        axs[0,0].plot(curr_Y_true_reg[:, coord], label = coords[coord])
      axs[1,0].plot(curr_Y_true_class[:, 0], label = 'true class')
      axs[1,0].plot(curr_Y_pred_class[:, 0], label = 'pred class')
      axs[0,0].legend()
      axs[0,0].set_title('right GRF')
      axs[1,0].legend()

      # plot left leg GRF
      for coord in range(3):
        axs[0,1].plot(curr_Y_true_reg[:, coord+3], label = coords[coord])
      axs[1,1].plot(curr_Y_true_class[:, 1], label = 'true class')
      axs[1,1].plot(curr_Y_pred_class[:, 1], label = 'pred class')
      axs[0,1].set_title('left GRF')

      # save fig
      plt.savefig(os.path.join(BASE_DIR, 'reports', 'figures', EXP_TYPE + '_lstm_stepclass', 'subj' + str(subj) + '_' + acl_tasks[task] + '_trial' + str(trial) + '.png'))


Output hidden; open in https://colab.research.google.com to view.